In [2]:
from src.datasets.novelty import *

In [1]:
from src.defaults import *

In [3]:
# download_models_from_neptune("NLI-81")
field = load_field('NLI-81')
# field = None


dataset_conf = {'dataset': 'dlnd', 'max_num_sent': 60,"sent_tokenizer":"spacy","batch_size":32,"device":"cuda"}
# dataset_conf = {'dataset': 'dlnd', 'max_num_sent': 50,"sent_tokenizer":"spacy", "tokenizer":'spacy',"max_len":50,"batch_size":32,"device":"cuda"}
model_conf = {'results_dir': 'results', 'device': 'cuda', 'dropout': 0.3, 'dataset': 'dlnd', 'hidden_size': 400, 'use_glove': False, "max_num_sent": 60}

In [4]:
from src.datasets.novelty import *
data = dlnd(dataset_conf,sentence_field = field)


In [5]:
for i in data.train_iter:
    print(i)
    break


[torchtext.data.batch.Batch of size 32]
	[.source]:[torch.cuda.LongTensor of size 32x60x50 (GPU 0)]
	[.target]:[torch.cuda.LongTensor of size 32x60x50 (GPU 0)]
	[.label]:[torch.cuda.LongTensor of size 32 (GPU 0)]


In [16]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class DAN(nn.Module):
    def __init__(self,conf,encoder):
        super(DAN,self).__init__()
        self.conf = conf
        self.num_sent = conf["max_num_sent"]
        self.encoder = encoder
        self.translate = nn.Linear(2 * self.conf["encoder_dim"], self.conf["hidden_size"])
        self.template = nn.Parameter(torch.zeros((1)), requires_grad=True)
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(conf["dropout"])

        self.mlp_f = nn.Linear(self.conf["hidden_size"], self.conf["hidden_size"])
        self.mlp_g = nn.Linear(2*self.conf["hidden_size"], self.conf["hidden_size"])
        self.mlp_h = nn.Linear(2*self.conf["hidden_size"], self.conf["hidden_size"])
        self.linear = nn.Linear(self.conf["hidden_size"],2)

    def encode_sent(self,inp):
        batch_size,num_sent,max_len = inp.shape
        x = inp.view(-1,max_len)

        x_padded_idx = x.sum(dim=1) != 0    
        x_enc = []
        for sub_batch in x[x_padded_idx].split(64):
            x_enc.append(self.encoder(sub_batch,None))
        x_enc = torch.cat(x_enc, dim=0)

        x_enc_t = torch.zeros((batch_size * num_sent, x_enc.size(1))).to(
            self.template.device
        )

        x_enc_t[x_padded_idx] = x_enc
        x_enc_t = x_enc_t.view(batch_size, num_sent, -1)
    
        embedded = self.dropout(self.translate(x_enc_t))
        embedded = self.act(embedded)
        return embedded


    def forward(self,x0,x1):
        x0_enc = self.encode_sent(x0)
        x1_enc = self.encode_sent(x1)

        f1 = self.act(self.dropout(self.mlp_f(x0_enc)))
        f2 = self.act(self.dropout(self.mlp_f(x1_enc)))

        score1 = torch.bmm(f1, torch.transpose(f2, 1, 2))
        prob1 = F.softmax(score1.view(-1, self.num_sent)).view(-1, self.num_sent, self.num_sent)

        score2 = torch.transpose(score1.contiguous(), 1, 2)
        score2 = score2.contiguous()

        prob2 = F.softmax(score2.view(-1, self.num_sent)).view(-1, self.num_sent, self.num_sent)

        sent1_combine = torch.cat((x0_enc, torch.bmm(prob1, x1_enc)), 2)
        sent2_combine = torch.cat((x1_enc, torch.bmm(prob2, x0_enc)), 2)

        

        g1 = self.act(self.dropout(self.mlp_g(sent1_combine)))
        g2 = self.act(self.dropout(self.mlp_g(sent2_combine)))

        sent1_output = torch.sum(g1, 1)  
        sent1_output = torch.squeeze(sent1_output, 1)
    
        sent2_output = torch.sum(g2, 1)  
        sent2_output = torch.squeeze(sent2_output, 1)


        input_combine = torch.cat((sent1_output * sent2_output, torch.abs(sent1_output - sent2_output)), 1)
        
        h = self.act(self.dropout(self.mlp_h(input_combine)))
        opt = self.linear(h)
        return opt
    

In [7]:
def load_encoder(_id):
    model_path = os.path.join('./results/',_id,"model.pt")
    model_data = torch.load(model_path)
    return model_data

In [8]:
model_data = load_encoder('NLI-81')

In [9]:
from src.model.nli_models import *

In [10]:
model_data['options']["use_glove"] = False
model = attn_bilstm_snli(model_data['options'])

In [11]:
model_conf["encoder_dim"] = model_data["options"]["hidden_size"]

In [12]:
def load_encoder1(enc_data):
    if enc_data['options'].get("attention_layer_param",0)==0:
        model = bilstm_snli(enc_data["options"])
    elif enc_data['options'].get("r",0)==0:
        model = attn_bilstm_snli(enc_data["options"])
    else:
        model = struc_attn_snli(enc_data["options"])
    return model
    

In [17]:
dan = DAN(model_conf,model.encoder)


In [18]:
dan = dan.cuda()
dan(i.source,i.target)

torch.Size([32, 60, 400])


tensor([[ 0.4049, -0.6092],
        [ 0.4137, -0.3006],
        [ 0.3718, -0.6981],
        [ 0.6268, -0.0797],
        [ 0.5702, -0.5165],
        [ 0.4372, -0.6110],
        [ 0.5662, -0.8030],
        [ 0.2436, -0.3077],
        [ 0.3847, -0.5314],
        [ 0.2711, -0.6244],
        [ 0.2584, -0.8445],
        [ 0.2959, -0.6516],
        [ 0.3599, -1.0253],
        [ 0.0326, -0.2944],
        [ 0.0797, -0.4814],
        [ 0.8330, -0.0779],
        [ 0.2417, -0.9891],
        [ 0.3088, -0.8536],
        [ 0.3115, -0.7002],
        [ 0.2796, -0.4201],
        [ 0.0923, -0.5198],
        [ 0.4817, -0.4856],
        [-0.2757, -0.5796],
        [ 0.5500, -0.5302],
        [ 0.4651, -0.4413],
        [ 0.3358,  0.0115],
        [ 0.5970, -0.5587],
        [ 0.5855, -0.9080],
        [ 0.0693, -0.8174],
        [ 0.4821, -0.4849],
        [ 0.6968, -0.8328],
        [ 0.3622, -0.5559]], device='cuda:0', grad_fn=<AddmmBackward>)